# Michigan FAP PDF → CSV (Multi-Year, Multi-County)

This notebook extracts *county-level monthly totals* tables from annual Michigan FAP PDFs where **one PDF per year contains all counties (~100 pages)**.

### What it does
- Detects **county names per page** via regex on page text (using `pdfplumber`).
- Extracts the **Monthly Totals** table from each page using **Tabula** (default) or **Camelot**.
- Cleans numbers (removes `$` and commas), standardizes column names, and keeps month rows.
- Writes **per-year CSVs** and an **all-years combined CSV**.

> **Note:** For scanned PDFs, please perform OCR (e.g., with Adobe Acrobat) before running this.


## 0) Install dependencies (run if needed)
Tabula requires Java (>=8). Camelot lattice mode requires ghostscript/opencv.


In [29]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk"
os.environ["PATH"] = "/opt/homebrew/opt/openjdk/bin:" + os.environ["PATH"]


In [ ]:
# If your environment misses these, uncomment and run:
# !pip install pdfplumber tabula-py camelot-py[cv] pandas python-dateutil tqdm
import sys
print(sys.version)

3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]
zsh:1: no matches found: camelot-py[cv]


In [12]:
!pip install "camelot-py[cv]" pdfplumber tabula-py pandas python-dateutil tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 37.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 27.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.8.10
    Uninstalling tabulate-0.8.10:
      Successfully uninstalled tabulate-0.8.10
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0
  A

In [9]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.9 MB/s eta 0:00:00a 0:00:01


In [18]:
import sys
!{sys.executable} -m pip install pdfplumber


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 52.2 MB/s  0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.1
    Uninstalling typing_extensions-4.13.1:
      Successfully uninstalled typing_extensions-4.13.132m0/7 [typing-extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pdfplumber]7 [pdfminer.six]


In [19]:
import pdfplumber, sys
print("pdfplumber:", pdfplumber.__version__, "| python:", sys.executable)


pdfplumber: 0.11.7 | python: /usr/local/bin/python3


In [20]:
import sys
!{sys.executable} -m pip install pdfplumber


In [21]:
import pdfplumber, sys
print("pdfplumber version:", pdfplumber.__version__)
print("python:", sys.executable)


pdfplumber version: 0.11.7
python: /usr/local/bin/python3


In [24]:
import sys
!{sys.executable} -m pip install tabula-py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 42.7 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tabula-py]


In [27]:
import os, subprocess

# 手动指定 Homebrew Java 路径
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk"
os.environ["PATH"] = "/opt/homebrew/opt/openjdk/bin:" + os.environ["PATH"]

# 验证是否生效
try:
    out = subprocess.check_output(["java", "-version"], stderr=subprocess.STDOUT).decode()
    print(out)
except subprocess.CalledProcessError as e:
    print(e.output.decode())



openjdk version "25" 2025-09-16
OpenJDK Runtime Environment Homebrew (build 25)
OpenJDK 64-Bit Server VM Homebrew (build 25, mixed mode, sharing)



In [28]:
import tabula
tabula.environment_info()   # 输出应含 “Java version: 25 …”


Python version:
    3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]
Java version:
    openjdk version "25" 2025-09-16
OpenJDK Runtime Environment Homebrew (build 25)
OpenJDK 64-Bit Server VM Homebrew (build 25, mixed mode, sharing)
tabula-py version: 2.10.0
platform: macOS-26.0.1-arm64-arm-64bit
uname:
    uname_result(system='Darwin', node='HUAWEI-MATEBOOK-PRO.local', release='25.0.0', version='Darwin Kernel Version 25.0.0: Wed Sep 17 21:38:03 PDT 2025; root:xnu-12377.1.9~141/RELEASE_ARM64_T8112', machine='arm64')
linux_distribution: ('Darwin', '25.0.0', '')
mac_ver: ('26.0.1', ('', '', ''), 'arm64')


## 1) Configure paths & options

In [2]:
import os
IN_DIR  = "/Users/jiamingzhang/Library/CloudStorage/OneDrive-SharedLibraries-MichiganStateUniversity/Wang, Hong Holly - Shared with Jiaming/SNAP/Data/Michigan FAP"
OUT_DIR = "/Users/jiamingzhang/Desktop/FAP_output"
os.makedirs(OUT_DIR, exist_ok=True)

START_YEAR = 2015
END_YEAR   = 2026
PER_COUNTY_CSV = True

# Ensure Java path (Apple Silicon default here)
import os as _os
_os.environ.setdefault("JAVA_HOME", "/opt/homebrew/opt/openjdk")
_os.environ["PATH"] = "/opt/homebrew/opt/openjdk/bin:" + _os.environ["PATH"]
print('OUT_DIR:', OUT_DIR)

OUT_DIR: /Users/jiamingzhang/Desktop/FAP_output


## 2) Helper: page area (county pages)

In [8]:
def page_area_points(pdf_path, page_no):
    import pdfplumber
    rel_area_county = (0.12, 0.06, 0.80, 0.96)  # try (0.10,0.06,0.84,0.96) if needed
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_no-1]
        H, W = page.height, page.width
    t = H*rel_area_county[0]; l = W*rel_area_county[1]; b = H*rel_area_county[2]; r = W*rel_area_county[3]
    return [t,l,b,r]

## 3) County detection (robust)

In [3]:
import re, os
def detect_counties_per_page(pdf_path, county_regex=None):
    import pdfplumber
    mi_counties = [
        'Presque Isle','Grand Traverse','St. Clair','St. Joseph','Van Buren',
        'Alcona','Alger','Allegan','Alpena','Antrim','Arenac','Baraga','Barry','Bay','Benzie',
        'Berrien','Branch','Calhoun','Cass','Charlevoix','Cheboygan','Chippewa','Clare','Clinton',
        'Crawford','Delta','Dickinson','Eaton','Emmet','Genesee','Gladwin','Gogebic',
        'Gratiot','Hillsdale','Houghton','Huron','Ingham','Ionia','Iosco','Iron','Isabella','Jackson',
        'Kalamazoo','Kalkaska','Kent','Keweenaw','Lake','Lapeer','Leelanau','Lenawee','Livingston',
        'Luce','Mackinac','Macomb','Manistee','Marquette','Mason','Mecosta','Menominee','Midland',
        'Missaukee','Monroe','Montcalm','Montmorency','Muskegon','Newaygo','Oakland','Oceana','Ogemaw',
        'Ontonagon','Osceola','Oscoda','Otsego','Ottawa','Roscommon','Saginaw','Sanilac',
        'Schoolcraft','Shiawassee','Tuscola','Washtenaw','Wayne','Wexford'
    ]
    mi_sorted = sorted(mi_counties, key=len, reverse=True)
    rx = re.compile(r"(?i)\b(?:St\.?\s+)?([A-Z][A-Za-z'\- ]+?)\s+County\b")
    page_to_county, last = {}, None
    print(f"[INFO] Scanning counties in {os.path.basename(pdf_path)} ...")
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ''
            name=None
            for c in mi_sorted:
                if re.search(rf"\b{re.escape(c)}\b", text, flags=re.I):
                    name=c; break
            if not name:
                m = rx.search(text)
                if m:
                    tmp = m.group(0); name = re.sub(r"(?i)\s+County\b","",tmp)
            if name:
                name = re.sub(r"\s+"," ",name).title()
                last = name
            page_to_county[i] = last
    print(f"[INFO] County detection complete. Total pages: {len(page_to_county)}")
    return page_to_county

## 4) Page extraction cascade

In [4]:
def read_tables_on_page(pdf_path, page_no, engine="tabula"):
    dfs = []
    area_pts = page_area_points(pdf_path, page_no)
    # Tabula lattice + area
    try:
        import tabula
        dfs = tabula.read_pdf(pdf_path, pages=str(page_no), lattice=True, multiple_tables=True, area=[area_pts], guess=False)
    except Exception:
        dfs = []
    # Tabula stream + area
    if not dfs:
        try:
            import tabula
            dfs = tabula.read_pdf(pdf_path, pages=str(page_no), lattice=False, multiple_tables=True, area=[area_pts], guess=False)
        except Exception:
            dfs = []
    # Camelot lattice
    if not dfs:
        try:
            import camelot
            tables = camelot.read_pdf(pdf_path, pages=str(page_no), flavor="lattice", strip_text=" \n")
            dfs = [t.df for t in tables if t.df is not None]
        except Exception:
            dfs = []
    # Camelot stream
    if not dfs:
        try:
            import camelot
            tables = camelot.read_pdf(pdf_path, pages=str(page_no), flavor="stream", strip_text=" \n")
            dfs = [t.df for t in tables if t.df is not None]
        except Exception:
            dfs = []
    return dfs or []

## 5) Normalize tables (merge two-row headers)

In [5]:
def normalize_table(df):
    import re, pandas as pd
    def _is_header_row(sr):
        txt = " ".join(map(lambda x: str(x).strip(), sr.tolist()))
        nonnum = sum(1 for v in sr if not re.fullmatch(r"\$?\s*[\d,\.\-]+", str(v).strip()))
        return nonnum >= max(3, int(0.6*len(sr))) and (re.search(r"Reporting|Month|Average|Recipients|Payments|Cases", txt, re.I) is not None)
    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all").copy()
    if df.shape[0] >= 2 and _is_header_row(df.iloc[0]) and _is_header_row(df.iloc[1]):
        new_cols = []
        for a,b in zip(df.iloc[0].astype(str), df.iloc[1].astype(str)):
            h = (a + " " + b).strip(); h = re.sub(r"\s+"," ",h); new_cols.append(h)
        df = df.iloc[2:].reset_index(drop=True); df.columns = new_cols
    else:
        if df.shape[0] > 0:
            first = df.iloc[0].astype(str).tolist()
            if _is_header_row(df.iloc[0]):
                df = df.iloc[1:].reset_index(drop=True); df.columns = first
    def canon(cols):
        mapping = {
            "reporting month":"Reporting Month","cases":"Cases","recipients":"Recipients",
            "adult recipients":"Adult Recipients","child recipients":"Child Recipients",
            "payments":"Payments","average per case":"Average Per Case","avg per case":"Average Per Case",
            "average per person":"Average Per Person","avg per person":"Average Per Person",
            "avg recipients per case":"Avg. Recipients Per Case","avg. recipients per case":"Avg. Recipients Per Case",
        }
        out=[]
        for c in cols:
            key = re.sub(r"[^a-z0-9 ]+","", str(c).strip().lower())
            out.append(mapping.get(key, str(c)))
        return out
    df.columns = canon(df.columns)
    keep = [c for c in ["Reporting Month","Cases","Recipients","Adult Recipients","Child Recipients","Payments","Average Per Case","Average Per Person","Avg. Recipients Per Case"] if c in df.columns]
    if not keep: return pd.DataFrame()
    df = df[keep].copy()
    if "Reporting Month" in df.columns:
        months = ["October","November","December","January","February","March","April","May","June","July","August","September","Monthly Avg"]
        pat = "|".join(map(re.escape, months))
        df["Reporting Month"] = df["Reporting Month"].astype(str).str.replace(r"\s+"," ", regex=True).str.strip()
        df = df[df["Reporting Month"].str.contains(pat, case=False, na=False)]
    def clean_number(x):
        if pd.isna(x): return pd.NA
        s = str(x).strip()
        if s in {"","—","-","NA","N/A"}: return pd.NA
        s = re.sub(r"[\$,]","",s); s = re.sub(r"[^0-9.\-]","",s)
        try: return float(s) if s else pd.NA
        except: return pd.NA
    for c in [c for c in df.columns if c != "Reporting Month"]:
        df[c] = df[c].apply(clean_number)
    return df

## 6) Process one year & batch

In [6]:
import re, glob, pandas as pd, os
def process_year_pdf(pdf_path, out_dir, year, write_per_county=False):
    page_to_county = detect_counties_per_page(pdf_path)
    rows = []
    for page_no, county in page_to_county.items():
        if page_no == 1:  # skip annual summary
            continue
        page_dfs = read_tables_on_page(pdf_path, page_no)
        total_rows = 0
        for idx, raw in enumerate(page_dfs):
            try:
                df = pd.DataFrame(raw); df = normalize_table(df)
                if df.empty: continue
                df["year"], df["county"], df["page_no"], df["table_index"] = year, county, page_no, idx
                rows.append(df); total_rows += len(df)
            except Exception:
                pass
        print(f"  [page {page_no:>2}] county={county or 'N/A'} rows={total_rows}")
    if not rows: return pd.DataFrame()
    out = pd.concat(rows, ignore_index=True).drop_duplicates()
    out["county"] = out["county"].ffill()
    if write_per_county:
        for cty, g in out.groupby("county", dropna=False):
            safe = "Unknown" if pd.isna(cty) else re.sub(r"[^A-Za-z0-9_]+","_", str(cty))
            g.to_csv(os.path.join(out_dir, f"FAP_{year}_{safe}.csv"), index=False)
    out_year_path = os.path.join(out_dir, f"FAP_{year}_ALL_COUNTIES.csv"); out.to_csv(out_year_path, index=False)
    return out

def run_batch(in_dir, out_dir, start_year, end_year, per_county_csv=True):
    os.makedirs(out_dir, exist_ok=True)
    pattern = os.path.join(in_dir, "FAP_Annual_*_Michigan_AllCounties.pdf")
    pdfs = sorted(glob.glob(pattern))
    if not pdfs: raise FileNotFoundError(f"No PDFs matched: {pattern}")
    combined = []
    for p in pdfs:
        base = os.path.basename(p)
        m = re.search(r"(?:FY)?(20\d{2})", base)
        if not m: 
            print(f"[WARN] Skip (no year): {base}"); continue
        year = int(m.group(1))
        if not (start_year <= year <= end_year): continue
        print(f"[INFO] Processing {base} (year={year})")
        dfy = process_year_pdf(p, out_dir, year, write_per_county=per_county_csv)
        if dfy is None or dfy.empty: 
            print(f"[WARN] No tables for {year}"); continue
        combined.append(dfy)
    if not combined:
        print("[WARN] Nothing extracted."); return pd.DataFrame()
    big = pd.concat(combined, ignore_index=True).drop_duplicates()
    big["county"] = big["county"].ffill()
    combined_path = os.path.join(out_dir, "FAP_ALL_YEARS_COMBINED.csv"); big.to_csv(combined_path, index=False)
    print(f"[DONE] Wrote {combined_path} rows={len(big)}"); return big

## 7) Quick single-year test (edit years if you like)

In [9]:
test_year_start, test_year_end = 2015, 2016
big_test = run_batch(IN_DIR, OUT_DIR, test_year_start, test_year_end, per_county_csv=True)
if isinstance(big_test, pd.DataFrame) and not big_test.empty:
    display(big_test.head(20))
    print('unique counties:', big_test['county'].nunique())
    print('pages captured:', big_test['page_no'].nunique())
    print('columns:', list(big_test.columns))

[INFO] Processing FAP_Annual_2015_Michigan_AllCounties.pdf (year=2015)
[INFO] Scanning counties in FAP_Annual_2015_Michigan_AllCounties.pdf ...


Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


[INFO] County detection complete. Total pages: 85
  [page  2] county=Alcona rows=13
  [page  3] county=Alger rows=13
  [page  4] county=Allegan rows=13
  [page  5] county=Alpena rows=13
  [page  6] county=Antrim rows=13
  [page  7] county=Arenac rows=13
  [page  8] county=Baraga rows=13
  [page  9] county=Barry rows=13
  [page 10] county=Bay rows=13
  [page 11] county=Benzie rows=13
  [page 12] county=Berrien rows=13
  [page 13] county=Branch rows=13
  [page 14] county=Calhoun rows=13
  [page 15] county=Cass rows=13
  [page 16] county=Charlevoix rows=13
  [page 17] county=Cheboygan rows=13
  [page 18] county=Chippewa rows=13
  [page 19] county=Clare rows=13
  [page 20] county=Clinton rows=13
  [page 21] county=Crawford rows=13
  [page 22] county=Delta rows=13
  [page 23] county=Dickinson rows=13
  [page 24] county=Eaton rows=13
  [page 25] county=Emmet rows=13
  [page 26] county=Genesee rows=13
  [page 27] county=Gladwin rows=13
  [page 28] county=Gogebic rows=13
  [page 29] county=Gra

,Cases,Recipients,Adult Recipients,Child Recipients,Payments,year,county,page_no,table_index
0,806.0,1479.0,1019.0,460.0,175887.0,2015,Alcona,2,0
1,797.0,1476.0,1013.0,463.0,174909.0,2015,Alcona,2,0
2,784.0,1460.0,1000.0,460.0,171119.0,2015,Alcona,2,0
3,789.0,1432.0,1004.0,428.0,165684.0,2015,Alcona,2,0
4,793.0,1447.0,1008.0,439.0,166483.0,2015,Alcona,2,0
5,786.0,1445.0,998.0,447.0,169121.0,2015,Alcona,2,0
6,795.0,1447.0,1006.0,441.0,170280.0,2015,Alcona,2,0
7,776.0,1423.0,989.0,434.0,166771.0,2015,Alcona,2,0
8,766.0,1393.0,979.0,414.0,159911.0,2015,Alcona,2,0
9,758.0,1392.0,969.0,423.0,155057.0,2015,Alcona,2,0


unique counties: 84
pages captured: 84
columns: ['Cases', 'Recipients', 'Adult Recipients', 'Child Recipients', 'Payments', 'year', 'county', 'page_no', 'table_index']


In [5]:
import pandas as pd
from pathlib import Path

# ========= Paths =========
base_dir = Path(r"/Users/jiamingzhang/Library/CloudStorage/OneDrive-SharedLibraries-MichiganStateUniversity/Wang, Hong Holly - Shared with Jiaming/SNAP/Data/Michigan FAP/FAP_output")
f1 = base_dir / "FAP_ALL_YEARS_COMBINED.csv"
f2 = base_dir / "FAP_ALL_YEARS_COMBINED2015-2016.csv"
f3 = base_dir / "FAP_2017_ALL_COUNTIES.csv"
out_path = base_dir / "FAP_ALL_YEARS_COMBINED_MERGED.csv"

# ========= 1) 读取并合并 =========
dfs = []
for p in [f1, f2, f3]:
    df = pd.read_csv(p, encoding="utf-8-sig", low_memory=False)
    # 统一列名小写
    df.columns = [c.strip().lower() for c in df.columns]
    # 标准化县名
    df["county"] = df["county"].astype(str).str.strip().str.title()
    # 年转整数
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    # 记录原始顺序，确保组内次序稳定
    df["_order"] = range(len(df))
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# ========= 2) 删除无效县名 =========
bad_pattern = r"^x-|unassigned|assigned"
df = df[~df["county"].str.contains(bad_pattern, case=False, regex=True, na=False)].copy()

# ========= 3) 删除平均/合计行 =========
# 你的文件没有显式“month”，每个 county×year 正常应有 12 行；
# 有些源文件第13行是 Average/Total。按“组内第13行”删除。
df = df.sort_values(["county", "year", "_order"]).copy()
df["_rank"] = df.groupby(["county", "year"]).cumcount() + 1
df = df[df["_rank"] <= 12].copy()

# ========= 4) 生成财政月(1..12)并映射到公历 =========
# 组内顺序就是 1..12：1=10月(上一年) … 3=12月(上一年) 4=1月(当年) … 12=9月(当年)
df["fiscal_month"] = df.groupby(["county", "year"])["_order"].rank(method="first").astype(int)

# 映射表：财政月 -> 公历月
F2C_MONTH = {1:10, 2:11, 3:12, 4:1, 5:2, 6:3, 7:4, 8:5, 9:6, 10:7, 11:8, 12:9}
df["month"] = df["fiscal_month"].map(F2C_MONTH).astype("Int64")

# 公历年：财政月 1–3(10–12月)属于上一年，其余属于当年
df["fy_orig"] = df["year"]
df["year"] = (df["fy_orig"] - (df["fiscal_month"] <= 3).astype("Int64")).astype("Int64")

# 生成公历日期
df["date_cal"] = pd.to_datetime(
    df["year"].astype(str) + "-" + df["month"].astype(str) + "-01",
    errors="coerce"
)

# ========= 5) 过滤到 2022 年及之前 =========
df = df[df["year"] <= 2022].copy()

# ========= 6) 去重：县×年×月 =========
df = df.sort_values(["county", "date_cal", "_order"])
df = df[~df.duplicated(["county", "year", "month"], keep="first")]

# ========= 7) 清理临时列并保存 =========
df = df.drop(columns=[c for c in ["page_no", "table_index", "_order", "_rank", "fiscal_month"] if c in df.columns])
front = ["county", "fy_orig", "year", "month", "date_cal"]
df = df[front + [c for c in df.columns if c not in front]]

df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("Saved to:", out_path)
print("Final shape:", df.shape)
print("Date range:", df["date_cal"].min(), "→", df["date_cal"].max())
print("Counties:", df["county"].nunique())
print(df.head(12)[["county","fy_orig","year","month","date_cal"]])


Saved to: /Users/jiamingzhang/Library/CloudStorage/OneDrive-SharedLibraries-MichiganStateUniversity/Wang, Hong Holly - Shared with Jiaming/SNAP/Data/Michigan FAP/FAP_output/FAP_ALL_YEARS_COMBINED_MERGED.csv
Final shape: (7968, 10)
Date range: 2014-10-01 00:00:00 → 2022-09-01 00:00:00
Counties: 83
      county  fy_orig  year  month   date_cal
6552  Alcona     2015  2014     10 2014-10-01
6553  Alcona     2015  2014     11 2014-11-01
6554  Alcona     2015  2014     12 2014-12-01
6555  Alcona     2015  2015      1 2015-01-01
6556  Alcona     2015  2015      2 2015-02-01
6557  Alcona     2015  2015      3 2015-03-01
6558  Alcona     2015  2015      4 2015-04-01
6559  Alcona     2015  2015      5 2015-05-01
6560  Alcona     2015  2015      6 2015-06-01
6561  Alcona     2015  2015      7 2015-07-01
6562  Alcona     2015  2015      8 2015-08-01
6563  Alcona     2015  2015      9 2015-09-01
